In [4]:
from syncode import Syncode

# 1. Define the grammar
Syncode allows user to define a grammar using a simple EBNF syntax adapted from Lark. The grammars for some common programming languages are defined in the `syncode/grammars` directory. One can also simply feed the grammar rules directly as a string of rules as shown below.

In [9]:
grammar = """
        start: expr

        ?expr: term
            | expr "+" term
            | expr "-" term
            | expr "*" term
            | expr "/" term
            | expr "=" term

        ?term: DEC_NUMBER | "(" expr ")"

        DEC_NUMBER: /0|[1-9]\d*/i
        
        %ignore " "
    """

Loading checkpoint shards: 100%|██████████████████████████████████████████████████████| 2/2 [00:00<00:00,  4.14it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading Lark base parser from cache: cache/parsers/custom_lalr_parser.pkl


# 2. Load the Huggingface models
Syncode uses the Huggingface transformers library to load the models. Simply provide the HuggingFace ID of the model to be used. Users can select the mode of generation `original` or `grammar_mask` to generate code with or without grammar constraining.

In [ ]:
model_name = "microsoft/phi-2"

# Load the unconstrained original model
llm = Syncode(model = model_name, mode='original', max_new_tokens=20, chat_mode=True)

# Load the Syncode augmented model
syn_llm = Syncode(model = model_name, mode='grammar_mask', grammar=grammar, chat_mode=True)

# 3. Comparing the outputs of Syncode with standard generation
We compare the outputs of Syncode with the standard generation. We see that by constraining the LLM generation with the arithmetic expression grammar, the LLM refrains from generating output that contains text and generates only arithmetic expressions. This is a powerful feature that can be used to generate syntacitcal correct code and outputs in specific contexts where the LLM output is automatically provided to another tool.

In [37]:
output = llm.infer("What is 2+2?")
print(f"LLM output:\n{repr(output)}\n")

output = syn_llm.infer("What is 2+2?")
print(f"Syncode augmented LLM output:\n{output}")

LLM output:
'The answer is 4.\n<|im_end|>\n\n<|im_start'

Syncode augmented LLM output:
2+2=4 


In [38]:
output = llm.infer('What is 7 multiplied by 8?')
print(f"LLM output:\n{repr(output)}\n")

output = syn_llm.infer('What is 7 multiplied by 8?')
print(f"Syncode augmented LLM output:\n{output}")

LLM output:
'The answer is 56.\n<|im_end|>\n\n<|im_start'

Syncode augmented LLM output:
7 * 8 = 56 


In [39]:
output = llm.infer('What is square root of 64?')
print(f"LLM output:\n{repr(output)}\n")

output = syn_llm.infer('What is square root of 64?')
print(f"Syncode augmented LLM output:\n{output}")

LLM output:
'The square root of 64 is 8.\n<|im_end|>\n'

Syncode augmented LLM output:
8 
